In [1]:
import awkward as ak
import torch.nn.functional as F
import numpy as np

from torch.utils.data import DataLoader
from glob import glob
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from models.particle_transformer import ParticleTransformer
from utils.data import Dataset, cycle, pad

In [3]:
features = ['PF_pt', 'PF_eta', 'PF_phi', 'PF_mass', 'PF_d0', 'PF_dz', 'PF_hcalFraction', 'PF_pdgId', 'PF_charge', 'PF_fromPV', 'PF_puppiWeightNoLep', 'PF_puppiWeight', 'PF_px', 'PF_py']
targets = ['truth_px', 'truth_py']

dataset = Dataset(
    glob('/ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/*.parquet'),
    features=features, targets=targets, start_ratio=0.0, end_ratio=0.001,
)

Loaded 113535 datapoints from /ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_10.parquet
Loaded 68904 datapoints from /ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_11.parquet
Loaded 104922 datapoints from /ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_12.parquet
Loaded 97875 datapoints from /ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_13.parquet
Loaded 52461 datapoints from /ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_14.parquet
Loaded 109620 datapoints from /ospool/cms-user/fudawei/DeepMETTraining/parquet/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-p

In [ ]:
dataloader = cycle(DataLoader(dataset=dataset, batch_size=256, shuffle=True, drop_last=True))
inputs, targets = next(dataloader)

In [ ]:
model = ParticleTransformer(
    input_dim=len(features),
    output_dim=len(targets),  
    # network configurations
    pair_input_dim=4,
    use_pre_activation_pair=False,
    embed_dims=[128, 512, 128],
    pair_embed_dims=[64, 64, 64],
    num_heads=8,
    num_layers=8,
    num_cls_layers=2,
    block_params=None,
    cls_block_params={'dropout': 0, 'attn_dropout': 0, 'activation_dropout': 0},
    fc_params=[],
    activation='gelu',
    # misc
    trim=True,
    for_inference=False,
)

In [8]:
outputs = model(inputs)

/home/fudawei/GenMET/models/particle_transformer.py:546: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.use_amp):


In [12]:
import torch